In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium.webdriver import Chrome
from selenium import webdriver
import json
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

In [2]:
name = ['017670','066570','139480','000270','000660','015760','069960','068270','105560','051910','090430','024110','012330','042660','006360','010950','097950','005930','005380','003490']
domain=[]
for i in name:
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ i + '&target=finsum_more'
    domain.append(base_url)
    print(domain)

['https://finance.naver.com/item/coinfo.nhn?code=017670&target=finsum_more']
['https://finance.naver.com/item/coinfo.nhn?code=017670&target=finsum_more', 'https://finance.naver.com/item/coinfo.nhn?code=066570&target=finsum_more']
['https://finance.naver.com/item/coinfo.nhn?code=017670&target=finsum_more', 'https://finance.naver.com/item/coinfo.nhn?code=066570&target=finsum_more', 'https://finance.naver.com/item/coinfo.nhn?code=139480&target=finsum_more']
['https://finance.naver.com/item/coinfo.nhn?code=017670&target=finsum_more', 'https://finance.naver.com/item/coinfo.nhn?code=066570&target=finsum_more', 'https://finance.naver.com/item/coinfo.nhn?code=139480&target=finsum_more', 'https://finance.naver.com/item/coinfo.nhn?code=000270&target=finsum_more']
['https://finance.naver.com/item/coinfo.nhn?code=017670&target=finsum_more', 'https://finance.naver.com/item/coinfo.nhn?code=066570&target=finsum_more', 'https://finance.naver.com/item/coinfo.nhn?code=139480&target=finsum_more', 'https:

In [90]:
browser  = webdriver.Chrome('C:/Users/12/Desktop/chromedriver_win32/chromedriver.exe')
name = ['017670','066570','139480','000270','000660','015760','069960','068270','105560','051910','090430','024110','012330','042660','006360','010950','097950','005930','005380','003490']
date=[]

for i in range(len(domain)):
    browser.get(domain[i])
    time.sleep(3)
    browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))# frame구조 
    time.sleep(1)
    browser.find_elements_by_xpath('//*[@class="schtab"][1]/tbody/tr/td[3]')[0].click()
    time.sleep(1)
    html0 = browser.page_source #지금 현 상태의 page source불러오기
    html1 = BeautifulSoup(html0,'html.parser')


    # 날짜 부분만 따로 저장
    #     date.append("".join(re.findall('[0-9/]',domain[i])))

    #기업 title불러오기
    title0 = html1.find('head').find('title').text
    title0 = title0.split('-')[-1]

    html22 = html1.find('table',{'class':'gHead01 all-width','summary':'주요재무정보를 제공합니다.'})
    thead0 = html22.find('thead') #날짜가 재무제표영역의 head부분에 들어가 있기 때문에 thead를 불러와야 한다.
    tr0 = thead0.find_all('tr')[1] #존재하고 있는 날짜대로 findall로 모두 수집
    th0 = tr0.find_all('th')
    date = []

    for i in range(len(th0)):
        date.append(''.join(re.findall('[0-9]', th0[i].text)))
    tbody0 = html22.find('tbody') #tbody에 column으로 사용할 데이터와 본문 데이터가 모두 담겨져 있다.
    tr0 = tbody0.find_all('tr')

    #columns 수집
    col = []
    for i in range(len(tr0)):

        if '\xa0' in tr0[i].find('th').text:
            tx = re.sub('\xa0','',tr0[i].find('th').text)
        else:
            tx = tr0[i].find('th').text

        col.append(tx)

    #본문데아터 수집
    td = []
    for i in range(len(tr0)):
        td0 = tr0[i].find_all('td')
        td1 = []
        for j in range(len(td0)):
            if td0[j].text == '':
                td1.append('0')
            else:
                td1.append(td0[j].text)
        td.append(td1)

    td2 = list(map(list,zip(*td)))
    td3 = pd.DataFrame(td2, columns = col, index = date)
    
    title0 = title0.split('-')[                                                                                            -1]
    filename = './financial_crawling/{}'.format(title0)

    td3.to_csv(filename)

    #     finance_df=pd.DataFrame(columns=['','','',''])

<ipython-input-90-4a6a7b3f9de2>:8: DeprecationWarning: use driver.switch_to.frame instead
  browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))# frame구조


In [88]:
td3 = pd.DataFrame(td2, columns = col, index = date)
title0 = title0.split('-')[-1]
filename = './{}'.format(title0)

td3.to_csv(filename)

In [91]:
filename

'./제무재표 크로울링/ 기업개요(대한항공)'

In [94]:
soil=pd.read_csv('C:/Users/12/Desktop/financial_crawling/기업개요(S-Oil)')

In [97]:
soil['PER(배)']

0     14.67
1      8.19
2     10.94
3     44.15
4    169.84
5       NaN
6     12.00
7     10.11
Name: PER(배), dtype: float64

In [100]:
soil[['PER(배)','PBR(배)','ROE(%)']]

,PER(배),PBR(배),ROE(%)
0,14.67,1.72,12.26
1,8.19,1.54,20.46
2,10.94,1.99,18.84
3,44.15,1.76,3.88
4,169.84,1.71,1.01
5,NaN,1.18,-13.95
6,12.00,1.09,9.46
7,10.11,1.01,10.41


In [33]:
cj=pd.read_csv("C:/Users/12/Desktop/재무제표 데이터/financial_crawling/기업개요(기업은행)")

In [34]:
cj[["Unnamed: 0",'ROE(%)','PER(배)','PBR(배)']]

,Unnamed: 0,ROE(%),PER(배),PBR(배)
0,201512,6.98,7.07,0.47
1,201612,6.59,7.21,0.47
2,201712,7.98,7.21,0.55
3,201812,8.63,5.27,0.44
4,201912,7.36,4.93,0.35
5,202012,6.12,4.41,0.28
6,202112,5.94,4.64,0.27
7,202212,6.03,4.35,0.26


In [8]:
cj[['unnamed','PER(배)','PBR(배)','ROE(%)']]

KeyError: "['unnamed'] not in index"